In [1]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import math
import sys
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle("./finalDF.pkl")

In [3]:
df

,year,price,distance,Origin_BOS,Origin_DTW,Origin_EWR,Origin_FLL,Origin_IAH,Origin_JFK,Origin_MIA,...,Month_Feb,Month_Jan,Month_Jul,Month_Jun,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,Month_nan
0,2020,91.16,946,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2020,118.67,946,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2020,199.67,946,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020,104.16,946,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020,113.38,946,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5929,2021,350.4,2393,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5930,2021,328.4,2393,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5931,2021,294.4,2393,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5932,2021,294.4,2393,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
y = df['price']
y

0        91.16
1       118.67
2       199.67
3       104.16
4       113.38
         ...  
5929     350.4
5930     328.4
5931     294.4
5932     294.4
5933     339.4
Name: price, Length: 5648, dtype: object

In [5]:
X = df.drop(columns=['price'])
X

,year,distance,Origin_BOS,Origin_DTW,Origin_EWR,Origin_FLL,Origin_IAH,Origin_JFK,Origin_MIA,Origin_MSP,...,Month_Feb,Month_Jan,Month_Jul,Month_Jun,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,Month_nan
0,2020,946,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2020,946,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2020,946,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020,946,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020,946,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5929,2021,2393,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5930,2021,2393,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5931,2021,2393,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5932,2021,2393,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
X_train, X_test, Y_train, Y_test, = train_test_split(X, y)

In [7]:
multivariate_linear = LinearRegression()

In [8]:
multivariate_linear.fit(X_train, Y_train)

LinearRegression()

In [9]:
y_train_pred = multivariate_linear.predict(X_train)

In [10]:
from sklearn.metrics import r2_score
r2_score(Y_train, y_train_pred)

0.20182310363584077

In [11]:
models = []
models.append(('LR', LogisticRegression(max_iter=100, multi_class='multinomial')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', weights='distance')))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='scale')))
models.append(('RF', RandomForestClassifier(criterion='gini', n_estimators=100, n_jobs=2)))

In [ ]:
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=7, random_state=1, shuffle=True)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

C:\Users\ekech\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\ekech\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
